**Learning Objectives:**
- Modeling of a Pickup-Delivery Vehicle Routing Problem(PDVRP-TW) on top of MDVRP from previous notebook
- Introduction to more advanced real world constraints

## Pickup and Delivery VRP with Time Windows (PDVRP-TW)

In scenarios where you need to do a series of pickup-delivery orders from multiple locations instead of pickup from single depot, we need to PDVRP-TW is used. 
A real world example would be, courier delivery companies are offering to pickup packages from customers and deliver them to recipients. 

**The company is now dealing with the following problem:**
- 98 locations in total:
    - 49 locations to pickup
    - 49 locations to delivery
    - 1 Depot (Distribution Center)
    
- 25 Vehicles in the Fleet
- Each vehicle has associated capacities
- Each location has a specific time window within which services/deliveries need to be performed
- Demand for orders may differ across locations
- Each vehicle has a time window of its availability

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies

from cuopt import routing
import cudf
import random
import cupy
import pandas as pd
from scipy.spatial import distance
import helper_function.helper_map as helper_map
from helper_function.helper_data import parse_output_summary, prepare_data_for_maps
import numpy as np
from helper_function.data_etl import get_minutes_from_datetime, build_distance_matrix, build_time_matrix, plot_order_locations
from helper_function.map_util import get_route, get_map, get_all_routes, get_map_by_vehicle

## Prepare Data

Remember the cuOpt rule to add depot locations at the beginning of the orders dataframe?

In [ ]:
# DO NOT CHANGE THIS CELL

DATA_PATH = "data/"

# load data

orders_details = pd.read_csv(DATA_PATH+"orders_pickup_delivery.csv")
depot_df       = pd.read_csv(DATA_PATH+"single_depot_location.csv")
n_depot        = len(depot_df)  

orders_details = pd.concat([orders_details, depot_df]).reset_index(drop=True)

orders_details.head()

In [ ]:
order_pickup_locations = pd.read_csv(DATA_PATH+"pickup_delivery_indices.csv")

In [ ]:
# DO NOT CHANGE THIS CELL
#pickup_locations = order_pickup_locations["pickup_locations"] + 1
#delivery_locations  = order_pickup_locations["delivery_locations"] + 1
pickup_locations = order_pickup_locations["pickup_locations"]
delivery_locations  = order_pickup_locations["delivery_locations"]
#n_locations      = len(pickup_locations) + len(order_locations)
#pickup_locations = random.choices(pickup_locations, k=len(order_locations))

In [ ]:
# DO NOT CHANGE THIS CELL
# Creating a 2-dimensional matrix from the columns "lat" and "lng" columns

location_coordinates = orders_details[['lat', 'lng']]
# Output looks like [[32.7727664105, -96.5800824456], [29.6832825242, -95.7305722105]............]


# Units of products to be delivered at each location, this can be number of orders, weight or volume.
order_demand               = orders_details['order_wt']
# 'order_demand' looks like [1020, 1989.......................2974]


### Preparing Time Windows

In [ ]:
# DO NOT CHANGE THIS CELL
orders_details['delivery_start_in_minutes']     = orders_details['delivery_start'].apply(get_minutes_from_datetime)
orders_details['delivery_end_in_minutes']       = orders_details['delivery_end'].apply(get_minutes_from_datetime)


# Earliest a delivery can be made
order_tw_earliest        = orders_details['delivery_start_in_minutes']
# Latest a delivery can be made
order_tw_latest          = orders_details['delivery_end_in_minutes']

# Service time required for the delivery/service
service_time         = orders_details['service_time']


In [ ]:
# DO NOT CHANGE THIS CELL

vehicle_df = pd.read_csv(DATA_PATH+"vehicles_all_multi_depot.csv")
vehicle_df['vehicle_start_in_minutes']     = vehicle_df['vehicle_start'].apply(get_minutes_from_datetime)
vehicle_df['vehicle_end_in_minutes']       = vehicle_df['vehicle_end'].apply(get_minutes_from_datetime)
vehicle_df.head()

In [ ]:
# DO NOT CHANGE THIS CELL

n_depot              = len(depot_df)
n_orders             = len(orders_details) - n_depot
n_locations          = len(location_coordinates)
n_vehicles           = len(vehicle_df)

# Number of orders each vehicle can carry, this can be number of orders, weight or volumne
vehicle_capacity     = vehicle_df['vehicle_capacity']

# Earliest a vehicle can start 
v_tw_earliest        = vehicle_df['vehicle_start_in_minutes']

# Latest a vehicle will be working
v_tw_latest          = vehicle_df['vehicle_end_in_minutes']

# Used to identify differnt vehicle
vehicle_colors       = ["blue", "white", "green", "pink", "yellow"]

In [ ]:
# DO NOT CHANGE THIS CELL

vehicle_df['vehicle_type_code'] = vehicle_df['vehicle_type'].astype('category')
vehicle_df['vehicle_type_code'] = vehicle_df['vehicle_type_code'].cat.codes

In [ ]:
# type 0 corresponds to bike and type 1 corresponds to car
vehicle_types = cudf.Series(vehicle_df['vehicle_type_code'])

### Visualize Depot and Destinations

This is an interactive map based on <mark>folium</mark> python mapping library

Depot: 🔴
 <br />Pickup: 🟢
 <br />Delivery: 🔵

In [ ]:
# DO NOT CHANGE THIS CELL

plot_order_locations(orders_details, location_coordinates, pdp=0)

In [ ]:
distance_matrix = build_distance_matrix(orders_details)
time_matrix = build_time_matrix(orders_details)

In [ ]:
# DO NOT CHANGE THIS CELL
# convert all data into cuDF dataframes
import cupy
distance_matrix = cudf.DataFrame(distance_matrix)
time_matrix = cudf.DataFrame(time_matrix)
pickup_locations = cudf.Series(pickup_locations)
delivery_locations  = cudf.Series(delivery_locations)

### Setting Order locations

From the cuOpt solver_settings perspective, each distinct transaction (pickup order or delivery order) is treated separately. The locations for each order is specified using order locations. The first entry in order locations array is always reserved for the notion of depot for the problem. So for a total n orders, the order location array is of size 2n+1.

In [ ]:
# DO NOT CHANGE THIS CELL

# concat order locations

pickup_order_locations = pd.concat([order_pickup_locations["pickup_locations"], order_pickup_locations["delivery_locations"]], ignore_index=True)

pickup_order_locations


### Mapping pickups to deliveries 

Order locations do not provide information regarding the type of order (i.e, pickup or delivery). This information is provided to solver by setting two arrays pickup_orders and delivery_orders. The entries of these arrays corresponding the order numbers in exanded list described above.

For a pair order i, pickup_orders[i] and delivery_orders[i] correspond to order index in 2n total orders. Furthermore, order_locations[pickup_orders[i]] and order_locations[delivery_orders[i]] indicate the pickup location and delivery location of order i.

When dropping off goods to the order locations, remove one unit of demand from the vehicle

In [ ]:
# Create pickup and delivery demand
# We already taken care of this step in data generation
order_demand

<br>

## Create Data-Model
---
Create a Data model with the following:
 - Number of locations
 - Number of vehicles in the fleet
 - Cost matrix
 - Location time windows
 - Vehicle time windows
 - Vehicle capacities
 - Variable demand across locations

In [ ]:
# DO NOT CHANGE THIS CELL

order_tw_earliest = cudf.Series(order_tw_earliest)

order_tw_latest = cudf.Series(order_tw_latest)

service_time = cudf.Series(service_time)

pickup_order_locations = cudf.Series(pickup_order_locations)

# We are making flexible time windows to help understand the concept when all pickup-delivery pairs are assigned

order_tw_earliest.iloc[:] = 480
order_tw_latest.iloc[:] = 1440

In [ ]:
# DO NOT CHANGE THIS CELL

data_model = routing.DataModel(n_locations, n_vehicles, n_orders)

data_model.add_cost_matrix(distance_matrix)

# add order locations
data_model.set_order_locations(pickup_order_locations)

# add pickup and delivery pairs.
data_model.set_pickup_delivery_pairs(cudf.Series(pickup_locations), cudf.Series(delivery_locations))

# Can you guess why we are using -1 in order_demand?

data_model.add_capacity_dimension(
    "order_wt",
    cudf.Series(order_demand[:-1]),
    cudf.Series(vehicle_capacity)
)
data_model.set_order_time_windows(
    cudf.Series(order_tw_earliest[:-1]),
    cudf.Series(order_tw_latest[:-1]), 
)

data_model.set_order_service_times(
    cudf.Series(service_time[:-1])
)

data_model.set_vehicle_time_windows(
    cudf.Series(v_tw_earliest), 
    cudf.Series(v_tw_latest)
)

# We are indicating that vehicle starts from depot
# Depot is at index 98 in the orders_details dataframe
# so we use 'n_orders' as index for specifying vehicle start & end location

data_model.set_vehicle_locations(
    cudf.Series([n_orders]*n_vehicles), cudf.Series([n_orders]*n_vehicles)
)
# Set minimum number of vehciles that need to be used to compute results
data_model.set_min_vehicles(5)

<br>

## Create Solver Instance
---
The solver instance will take the data-model and return an optimized route plan. Additional configuration options are available to further customize solver behavior including: 
- The number of parallel agents (climbers) examining the solution search space
- The maximum time allotted to find a solution
- The minimum number of vehicles to be used
- and more

In [ ]:
# DO NOT CHANGE THIS CELL
solver_settings = routing.SolverSettings()
# set number of climbers that will try to search for an optimal path parallely
solver_settings.set_number_of_climbers(4096)
# solver will run for given time limit and it will fail if needs more time
solver_settings.set_time_limit(5)

solver_settings.set_drop_infeasible_orders(True)

routing_solution = routing.Solve(data_model, solver_settings)
if routing_solution.get_status() == 0:
    print("Solution Found")
else:
    print("No Solution Found")

**Infeasible Solve** means, there is not a solution that can assign all available orders to vehicles available. There are two possible steps you can consider now. 

1. Relax time constraint by setting the scope to SOFT_TW will expand the search to unfeasible regions. Currently the SOFT_TW scope only relaxes the latest time. The solution can still be unfeasible due to demand and slack time constraints. [More details](https://docs.nvidia.com/cuopt/py_api.html#cuopt.routing.SolverSettings.set_solution_scope)

2. Second approach is to use `solver_settings.set_drop_infeasible_orders(True)` and let the solver find a solution.


In [ ]:
# DO NOT CHANGE THIS CELL

routing_solution = routing.Solve(data_model, solver_settings)
if routing_solution.get_status() == 0:
    print("Solution Found")
    output = parse_output_summary(routing_solution, n_orders+n_depot, n_vehicles, orders_details, vehicle_df, location_coordinates, distance_matrix, time_matrix, n_depot)
else:
    print("No Solution Found")


In [ ]:
routes_df = routing_solution.get_route().to_pandas()
routes_df[routes_df['type']=='Depot']


In [ ]:
plot_map_df = prepare_data_for_maps(depot_df, output)
all_routes = get_all_routes(plot_map_df)

In [ ]:
# DO NOT CHANGE THIS CELL

from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options=list(vehicle_df.vehicle_id),
    value=list(vehicle_df.vehicle_id)[0],
    description='Vehicle ID:',
)

def on_change(value):
    if value in output.keys():
        idx = list(plot_map_df[plot_map_df['vehicle_id'] == value].index)
        filtered_dict = {k: v for k, v in all_routes.items() if k in idx}
        display(get_map_by_vehicle(filtered_dict))
        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w)